<a href="https://colab.research.google.com/github/kupieckozienice/pyPDF2/blob/main/pypdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LIBRARY] zainstalowanie biblioteki PyPDF2

In [52]:
# Clone the entire repo.
!git clone -l -s https://github.com/kupieckozienice/pyPDF2 cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 8), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (27/27), 14.15 KiB | 14.15 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/cloned-repo
AAA  pypdf.ipynb  pyPDF.ipynb


In [40]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.8 MB/s eta 0:00:00


In [39]:
pip install PyPDF2

SyntaxError: invalid syntax (<ipython-input-39-a3ad871f8690>, line 1)

[LIBRARY] install pdfminer.six

In [3]:
pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.4 MB/s eta 0:00:00


[LIBRARY] importowanie niezbędnych *bibliotek*

In [19]:
import os
import pathlib as pth
from typing import Union, Literal, List
from PyPDF2 import PdfReader, PdfWriter

[DATA] konwersja daty

In [20]:
def convertPdfDatetime(pd):
    from datetime import datetime
    dtformat = "%Y%m%d%H%M%S"
    clean = pd.replace("D:","").split('+')[0]
    return datetime.strptime(clean,dtformat)

[PDF] add blank page

In [21]:
# add blank page
def add_blank_page(writer, path_destination):
  writer.add_blank_page(width=200, height=200)

  data = b"any bytes - typically read from a file"
  writer.add_attachment("smile.png", data)

  with open("output.pdf", "wb") as output_stream:
      writer.write(output_stream)

[PDF] sprawdzenie czy jest zaszyfrowany hasłem,
jeśli tak to należy do odszyfrować pobranym hasłem



In [22]:
def check_if_encrypted(path_to_file):
  if os.path.isfile(path_to_file):
    reader = PdfReader(path_to_file)

    if reader.is_encrypted:
      writer = PdfWriter()
      password = input("Enter your password: ")
      reader.decrypt(password)

      # Add all pages to the writer
      for page in reader.pages:
          writer.add_page(page)

      # Save the new PDF to a file
      with open(path_to_file, "wb") as f:
          writer.write(f)

      print("plik rozszyfrowany")
    else:
      print("plik NIE był zaszyfrowany")

[PDF] czytanie adnotacji

In [23]:
def annots_reading(reader):
  print("funkcja annots_reading załadowana")

  attachments = {}

  for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            obj = annot.get_object()
            annotation = {"subtype": obj["/Subtype"], "location": obj["/Rect"]}
            print(annotation)

            # text
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Text":
                print(annot.get_object()["/Contents"])

            # Highlights
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Highlight":
                coords = annot.get_object()["/QuadPoints"]
                x1, y1, x2, y2, x3, y3, x4, y4 = coords

            # # attachments
            # subtype = annot.get_object()["/Subtype"]
            # if subtype == "/FileAttachment":
            #     fileobj = annotobj["/FS"]
            #     attachments[fileobj["/F"]] = fileobj["/EF"]["/F"].get_data()
    else:
      # print("brak andotacji w pliku")
      pass

[PYTHON] iterate over files in directory

In [24]:
# iterate over files in directory

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

[PDF] add the metadata

In [25]:
# Add the metadata
def add_metadata(path_to_file):
  reader = PdfReader(path_to_file)
  writer = PdfWriter()

  # Add all pages to the writer
  for page in reader.pages:
      writer.add_page(page)

  writer.add_metadata(
      {
          "/Author": "Martino",
          "/Producer": "Libre Writer",
          "/Subject": "kikk",
          "/Title": "jakis_tytul",
          # "/ModDate": modification_date  # Adding modification date
      }
  )

  with open(os.path.join(path_to_working_directory, path_to_file), "wb") as f:
    writer.write(f)


[PDF] read metadata

In [26]:
def read_metadata(reader):
  meta = reader.metadata
  print(len(reader.pages))

  # All of the following could be None!
  print(f"author: {meta.author}")
  print(f"creator: {meta.creator}")
  print(f"producer: {meta.producer}")
  print(f"subject: {meta.subject}")
  print(f"title: {meta.title}")

[PDF] reading form fields

In [27]:
# reading form fields

def reading_form_fields(reader):
  # fields = reader.get_form_text_fields()
  # fields == {"key": "value", "key2": "value2"}
  # if reader.isEditable:
  fields = reader.get_fields()

  if bool(fields):
    # print(fields)
    for field_name, field_value in fields.items():
      print(f"nazwa pola: {field_name}, wartość: {field_value}")
  else:
    print("brak fields-ów")

  # for field_name in fields:
    # print(field_name)

[PDF] extract text

In [28]:
# extract text
def extract_text(reader):
  text = ""
  try:
      for page in reader.pages:
          text += page.extract_text()
  except Exception as exc:
      text = fallback_text_extraction("example.pdf")

  return text

[PYTHON] sprawdzenie czy istnieje katalog roboczy

In [29]:
# sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować
def check_if_directory_and_path_exists(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

[PYTHON] upload plików z dysku lokalnego

In [30]:
def import_files(destination_directory):
  from google.colab import files

  uploaded = files.upload()
  # print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory) and os.path.isdir(destination_directory):
      with open(os.path.join(destination_directory, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")



[pdf] usuwanie stron w pdf

In [31]:
# usuwanie stron w pdf
def page_deleting(source_file, pages_nums_to_delete):
  print(source_file)

  file_name_without_ext, ext = os.path.splitext(source_file)
  new_file_name = ''.join([os.path.basename(file_name_without_ext), ext])
  # new_file_name = ''.join([os.path.basename(file_name_without_ext), "_with_deleted_pages", ext])

  destination_file = os.path.join(os.path.dirname(source_file), new_file_name)
  print(file_name_without_ext)
  print(ext)
  print(new_file_name)

  with open(source_file, "rb") as file:
    reader = PdfReader(file)
    writer = PdfWriter()

    for page_num in range(len(reader.pages)):
      if page_num + 1 not in pages_nums_to_delete:
        page_to_stay = reader.pages[page_num]
        writer.add_page(page_to_stay)

  with open(destination_file, "wb") as new_file:
    writer.write(new_file)

  with open(destination_file, "rb") as new_file:
    print(f"liczba stron nowego pliku: {len(PdfReader(new_file).pages)}")


In [32]:
# check if file exists
def check_if_file_exists(path_to_file):
  try:
    with open(path_to_file, 'r') as file:
      print(f"Plik {path_to_file} istnieje i ma się całkiem nieźle ;-)")
      return True
  except FileNotFoundError:
    print('Sorry the file we\'re looking for doesn\' exist')
    return False
  except Exception as e:
    print(f"Wystąpił błąd: {e}")
    return False

main app

In [ ]:
from datetime import datetime  # Import the datetime module
from pdfminer.high_level import extract_text as fallback_text_extraction

# Ścieżka do katalogu, w którym będą uploudowene pliki
path_to_working_directory = os.path.join(os.getcwd(), "pypdf2")
print(f"Ścieżka do katalogu, w którym będą uploudowene pliki: {path_to_working_directory}")

# sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować
check_if_directory_and_path_exists(path_to_working_directory)

# nazwa pliku roboczego wraz ze ścieżką
filename = os.path.join(os.getcwd(), "pypdf2", "oferta_handlowa_wzor.pdf")
check_if_file_exists(filename)

# usuwanie stron
page_to_delete = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
page_deleting(filename, page_to_delete)

reader = PdfReader(filename)
writer = PdfWriter()

check_if_encrypted(filename)

# annots reading
annots_reading(reader)

# czytanie metadanych
meta = reader.metadata
meta

# iterate over files in directory
iterate_over_files_in_directory(path_to_working_directory)

# Reading form fields
reading_form_fields(reader)

# extract text
text = extract_text(reader)
print(text)

# add metadata
# add_metadata(filename)

In [ ]:
# upload files
path_to_uploading_directory = os.path.join(os.getcwd(), "uploaded")
check_if_directory_and_path_exists(path_to_uploading_directory)
import_files(path_to_uploading_directory)

pydocx

In [83]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from google.colab import files
from datetime import datetime

doc = Document()

# modyfikacja stylu Normal
normal_style = doc.styles['Normal']
normal_style.font.name = 'Tahoma'
normal_style.font.size = Pt(9)
normal_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT

# styl nagłówka GŁÓWNEGO
style = doc.styles.add_style('nagłówek_główny_styl', 1)
style.font.name = 'Tahoma'
style.font.size = Pt(9)
style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

# zmiana marginesu storny
sekcja = doc.sections[0]
sekcja.left_margin = Inches(1)
sekcja.right_margin = Inches(1)
szerokosc_strony = sekcja.page_width - sekcja.left_margin - sekcja.right_margin

# dodanie logo firmy
logo_sciezka = '/content/uploaded/nagłowek_enea.PNG'
logo_added = doc.add_picture(logo_sciezka)
wciecie_logo = szerokosc_strony - logo_added.width / 2

# dane wejściowe
nazwa_sprawy = "KPI-2023.ŁA.23.2024"
data_dokumentu = datetime.now().strftime('%d.%m.%Y')

# dodanie znaku sprawy i daty
p = doc.add_paragraph(f'Nasz znak: {nazwa_sprawy}')
p.style = 'Normal'

data = doc.add_paragraph(f"Świerże Górne, dn. {data_dokumentu}r.")
data.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# dodanie nagłówka
nagłówek_text = doc.add_heading('ZAPYTANIE OFERTOWE (RFX)', 0)
nagłówek_text.style = 'nagłówek_główny_styl'



# document.add_heading('Heading, level 1', level=1)
# document.add_paragraph('Intense quote', style='Intense Quote')

# document.add_paragraph(
#     'first item in unordered list', style='List Bullet'
# )

pkt_1 = "WPROWADZENIE:\nNiniejsze Zapytanie jest podstawą do złożenia ofert."
pkt_2 = "TRYB UDZIELENIA ZAMÓWIENIA:\nPostępowanie w trybie Zapytania Ofertowego (RFX) prowadzone jest w oparciu o wewnętrzny Regulamin Udzielania Zamówień i Zawierania Umów (RUZiZU) Zamawiającego, dla którego nie mają zastosowania przepisy ustawy z dnia 11.09.2019 r. Prawo zamówień publicznych."
podpunkt_3_1 = f"Wykonanie usługi pn. \"{nazwa_sprawy}\"."
podpunkt_3_2 = "Zakres dostawy wg: Załącznika Nr 1 pn. \"Zakres Przedmiotu zamówienia\"."
pkt_3 = {"PRZEDMIOT ZAMÓWIENIA": [podpunkt_3_1, podpunkt_3_2]}
lista_punktów = [pkt_1, pkt_2, pkt_3]

for punkt in lista_punktów:
  if isinstance(punkt, dict):
    for main_item, sub_items in punkt.items():
      paragraf = doc.add_paragraph(main_item, style = 'ListNumber')
      for sub_item in sub_items:
        sub_paragraf = doc.add_paragraph(sub_item, style = 'ListBullet')
        sub_paragraf.paragraph_format.left_indent = 360
  else:
    paragraf = doc.add_paragraph(punkt, style = 'ListNumber')

# ustalenie odstępów między punktami
# paragraf.paragraph_format.space_after = 100000
# paragraf.paragraph_format.space_before = 100


# jeden = doc.add_paragraph(
#   'WPROWADZENIE:\n', style='List Number',
# )

# jeden.add_run('Niniejsze Zapytanie jest podstawą do złożenia ofert.')
# jeden.add_run('bold').bold = True
# jeden.add_run('italic.').italic = True


# records = (
#     (3, '101', 'Spam'),
#     (7, '422', 'Eggs'),
#     (4, '631', 'Spam, spam, eggs, and spam')
# )

# table = document.add_table(rows=1, cols=3)
# hdr_cells = table.rows[0].cells
# hdr_cells[0].text = 'Qty'
# hdr_cells[1].text = 'Id'
# hdr_cells[2].text = 'Desc'
# for qty, id, desc in records:
#     row_cells = table.add_row().cells
#     row_cells[0].text = str(qty)
#     row_cells[1].text = id
#     row_cells[2].text = desc

# document.add_page_break()

doc.save('demo.docx')
files.download('demo.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>